In [ ]:
import pandas as pd
import sqlite3 


In [ ]:
def rrsql(query):
    return pd.read_sql_query(query,conn)

In [ ]:
df=pd.read_csv("Churn_Modelling.csv")
df.info()

In [ ]:
df.columns

In [ ]:
df['Exited'] = df['Exited'].map({1: 'Yes', 0: 'No'})
df['IsActiveMember'] = df['IsActiveMember'].map({1: 'Yes', 0: 'No'})
df['HasCrCard'] = df['HasCrCard'].map({1: 'Yes', 0: 'No'})

In [ ]:
customers_df=df[['CustomerId', 'Surname','Gender', 'Age','Geography']].drop_duplicates().reset_index(drop=True)

In [ ]:
accounts_df=df[['CustomerId','CreditScore','Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary']].drop_duplicates().reset_index(drop=True)

In [ ]:
status_df=df[["CustomerId",'Exited']].reset_index()
status_df.rename(columns={"index":"StatusID"},inplace=True)
status_df

In [ ]:
conn=sqlite3.connect("Model.db")

In [ ]:
c=conn.cursor()

In [ ]:
c.execute("""CREATE TABLE IF NOT EXISTS customer(
"CustomerId" TEXT PRIMARY KEY UNIQUE,'Surname' TEXT,'Gender' TEXT,
'Age' INT,'Geography' TEXT)""")

In [ ]:
c.execute("""CREATE TABLE IF NOT EXISTS account(
'CustomerId' TEXT,'CreditScore' INT,'Tenure' INT, 'Balance' INT,
'NumOfProducts' INT, 'HasCrCard' TEXT,
'IsActiveMember' TEXT, 'EstimatedSalary' INT,
FOREIGN KEY (CustomerId) REFERENCES customer(CustomerId))""")

In [ ]:
c.execute("""CREATE TABLE IF NOT EXISTS status(
"StatusID" INT PRIMARY KEY, "CustomerId" TEXT, "Exited" Text,
FOREIGN KEY (CustomerId) REFERENCES customer(CustomerId))""")

In [ ]:
customers_df.to_sql("customer",conn,if_exists="replace",index=False)
accounts_df.to_sql("account",conn,if_exists="replace",index=False)
status_df.to_sql("status",conn,if_exists="replace",index=False)

In [ ]:
# 1. Customers who have exited, sorted by highest balance
rrsql("""
SELECT c.CustomerId, c.Surname, a.Balance
FROM customer c
JOIN account a ON c.CustomerId = a.CustomerId
JOIN status s ON c.CustomerId = s.CustomerId
WHERE s.Exited = 'Yes'
ORDER BY a.Balance DESC;
""")

In [ ]:
# 2. Count of active vs inactive members
rrsql("""
SELECT a.IsActiveMember, COUNT(*) AS Count
FROM account a 
GROUP BY a.IsActiveMember;
""")

In [ ]:
 # 3. Customers without a credit card
rrsql("""
SELECT c.CustomerId, c.Surname
FROM customer c
JOIN account a ON c.CustomerId = a.CustomerId
WHERE a.HasCrCard = 'No';
""")

In [ ]:
# 4. Exited customers from each geography
rrsql("""
SELECT c.Geography, COUNT(*) AS ExitedCount
FROM customer c
JOIN status s ON c.CustomerId = s.CustomerId
WHERE s.Exited = 'Yes'
GROUP BY c.Geography;
""")

In [ ]:
# 5. Customers with salary above 100000
rrsql("""
SELECT c.CustomerId, c.Surname, a.EstimatedSalary
FROM customer c
JOIN account a ON c.CustomerId = a.CustomerId
WHERE a.EstimatedSalary > 100000;
""")

In [ ]:
# 6. Tenure distribution for customers who exited
rrsql("""
SELECT a.Tenure, COUNT(*) AS CustomerCount
FROM account a
JOIN status s ON a.CustomerId = s.CustomerId
WHERE s.Exited = 'Yes'
GROUP BY a.Tenure
ORDER BY a.Tenure;
""")


In [ ]:
# 7. Average credit score by geography
rrsql("""
SELECT c.Geography, AVG(a.CreditScore) AS AvgCreditScore
FROM customer c
JOIN account a ON c.CustomerId = a.CustomerId
GROUP BY c.Geography;
""")

In [ ]:
# 8. Customers aged above 50 who are active members
rrsql("""
SELECT c.CustomerId, c.Surname, c.Age, a.IsActiveMember
FROM customer c
JOIN account a ON c.CustomerId = a.CustomerId
WHERE c.Age > 50 AND a.IsActiveMember ="Yes";
""")

In [ ]:
# 9. Average balance by geography
rrsql("""
SELECT c.Geography, AVG(a.Balance) AS AvgBalance
FROM customer c
JOIN account a ON c.CustomerId = a.CustomerId
GROUP BY c.Geography;
""")

In [ ]:
# 10. Average credit score of active vs inactive member
rrsql("""
SELECT a.IsActiveMember, AVG(a.CreditScore) AS AvgCreditScore
FROM account a
GROUP BY a.IsActiveMember;
""")


In [ ]:
# 11.Find the top 10 customers with the highest balance who are still active members and haven’t exited
rrsql("""
SELECT c.CustomerId, c.Surname, a.Balance, a.IsActiveMember, s.Exited
FROM customer c
JOIN account a ON c.CustomerId = a.CustomerId
JOIN status s ON c.CustomerId = s.CustomerId
WHERE a.IsActiveMember = "Yes" AND s.Exited ="No"
ORDER BY a.Balance DESC
LIMIT 10;
""")


In [ ]:
# 12.Find customers older than 50 years with a credit score below 600

rrsql("""
SELECT c.CustomerId, c.Surname, c.Age, a.CreditScore
FROM customer c
JOIN account a ON c.CustomerId = a.CustomerId
WHERE c.Age > 50 AND a.CreditScore < 600
ORDER BY a.CreditScore ASC;
""")